# Banking Chatbot Simulator

This Jupyter Notebook demonstrates the use of a banking chatbot simulator. The notebook is designed to simulate interactions with a banking chatbot to resolve various banking-related queries. The main components of the notebook include:

1. **CopilotStudioBot Class**: This class handles the interaction with the chatbot, including starting a conversation, sending questions, and receiving responses.

2. **UserSimulator Class**: This class simulates a user interacting with the chatbot. It uses the Azure OpenAI service to generate the next question to ask the chatbot based on the chatbot's response.

3. **Environment Setup**: The notebook uses environment variables to configure the bot name and environment name. These variables are used to initialize the `CopilotStudioBot` and `UserSimulator` classes.

4. **Conversation Simulation**: The notebook contains several cells that simulate conversations with the chatbot. Each cell demonstrates a different query being asked to the chatbot and the response received.

## Key Variables

- **BOT_NAME**: The name of the chatbot.
- **ENV_NAME**: The environment name where the chatbot is hosted.
- **PROMPT**: The initial prompt provided to the Azure OpenAI service to guide the conversation.
- **client**: An instance of the Azure OpenAI client used to generate the next question.
- **result**: The result of the conversation simulation, indicating whether the conversation was successful or if it required escalation to a human.
- **simulator**: An instance of the `UserSimulator` class used to simulate the conversation.

## Usage

To use this notebook, simply run each cell in sequence. The notebook will simulate interactions with the banking chatbot and display the results of each conversation. The conversations are designed to stop as soon as the initial question is answered or if the user needs to speak to a human.

In [1]:

import pandas as pd
import requests
import json
import time
import json

class CopilotStudioBot:

    """
    "This class is used to interact with the Copilot Studio Bot API."
    """
    
    def __init__(self, bot_name, env_name):
        self.bot_name = bot_name
        self.env_name = env_name
        self.token = None
        self.last_conversation_id = None

    def start_conversation(self):
        """
        Starts a new conversation with the Copilot Studio Bot.

        This method first retrieves an authentication token for the bot, then uses this token to create a new conversation.
        The conversation ID is stored in the instance variable `last_conversation_id`.

        Returns:
            str: The ID of the newly created conversation.
        """

        url = f"https://{self.env_name}.environment.api.powerplatform.com/powervirtualagents/botsbyschema/{self.bot_name}/directline/token"
        params = {"api-version": "2022-03-01-preview"}

        response = requests.get(url, params=params)

        # Extract the token from the response
        self.token = response.json().get('token')

        # Create a new conversation
        url = "https://directline.botframework.com/v3/directline/conversations"
        headers = {"Authorization": f"Bearer {self.token}"}

        response = requests.post(url, headers=headers)

        # Extract the conversation_id from the response
        conversation_id = response.json().get('conversationId')
        self.last_conversation_id = conversation_id

        return conversation_id

    def send_question(self, question, conversation_id= None, verbose = 0):
        """
        Sends a question to the Copilot Studio Bot and retrieves the response.

        This method sends a question to the bot using the Direct Line API and waits for the bot's response.
        If no conversation ID is provided, it starts a new conversation.

        Args:
            question (str): The question to send to the bot.
            conversation_id (str, optional): The ID of the conversation to use. If None, a new conversation is started. Defaults to None.
            verbose (int, optional): The verbosity level. If greater than 0, prints the response. If greater than 1, prints detailed response information. Defaults to 0.

        Returns:
            tuple: A tuple containing the bot's response (str) and the conversation ID (str).
        """
        
        if self.token is None or conversation_id is None:
            self.last_conversation_id = self.start_conversation()
            if self.last_conversation_id is None:
                return "Failed to start conversation"
            conversation_id = self.last_conversation_id


        # Send a message to the bot
        url = f"https://directline.botframework.com/v3/directline/conversations/{conversation_id}/activities"
        headers = {
            "Authorization": f"Bearer {self.token}",
            "Content-Type": "application/json"
        }

        # input_text = ""
        # expected_response = ""
        # for activity in conversation['Activities']:
        #     if activity['From']['Role'] == 1:

        body = {
            "locale": "en-US",
            "type": "message",
            "from": {"id": "user1"},
            "text": question
        }

        response = requests.post(url, headers=headers, data=json.dumps(body))

        if verbose:
            print(response.json())
        message_id = response.json().get('id')

        str_response = ''
        messages_read = []
        tries = 0
        # Keep checking for the bot's response until it's received
        while tries < 20:
            response = requests.get(url, headers=headers)
            if verbose > 1:
                print(response.json())
            activities = response.json().get('activities', [])
            for activity in activities:  # Start from the end of the list
                if activity.get('replyToId') == message_id and activity.get('id') not in messages_read:  # Check if the activity is a response to your message
                    try: 
                        role = activity.get('from',{}).get('role')
                        if role == 'bot':
                            bot_message = activity.get('text')
                            if verbose > 1:
                                print(activity)
                            if verbose:
                                print(activity.get('text'), activity.get('suggestedActions',""))
                            str_response += f"{bot_message}\n"
                            messages_read.append(activity.get('id') )
                    except:
                        print('\nNB==> Could not work out the role, discarded\n')
    
            if tries > 10 and len(str_response) > 0:
                break
            tries += 1
            time.sleep(.5)

        return str_response, conversation_id

    def restart_conversation(self,question,verbose=0):
        """
        Restarts the conversation with the Copilot Studio Bot.

        This method sends a question to the bot and retrieves the response, effectively restarting the conversation.

        Args:
            question (str): The question to send to the bot.
            verbose (int, optional): The verbosity level. If greater than 0, prints the response. If greater than 1, prints detailed response information. Defaults to 0.

        Returns:
            tuple: A tuple containing the bot's response (str) and the conversation ID (str).
        """
        return self.send_question(question,verbose=verbose)


    def disambig(self,bot_response):
        """
        Disambiguates the bot's response to extract the most relevant intent.

        This method attempts to parse the bot's response as JSON and extract the intents.
        It sorts the intents by their score in descending order and returns the display name of the highest-scoring intent,
        excluding any intent with the display name 'None of these'. If parsing fails, it returns the raw bot response.

        Args:
            bot_response (str): The response from the bot.

        Returns:
            str: The display name of the highest-scoring intent or the raw bot response if parsing fails.
        """
        try: 
            json_response = json.loads(bot_response)
            intents = [j["DisplayName"] for j in json_response]
            sorted_response = sorted(json_response, key=lambda x: x['Score'], reverse=True)
            sorted_response = [item for item in sorted_response if item['DisplayName'] != 'None of these']
            return sorted_response[0]['DisplayName'].replace("\n","")
        except:
            return bot_response.replace("\n","")

In [2]:
# call open ai completion 1.0

import os
from openai import AzureOpenAI

from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version = os.getenv("AZURE_OPENAI_API_VERSION"),
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

PROMPT = '''
You are a banking customer. 

You will ask an external banking chatbot to help you with your banking needs.

You will be provided with a question, that you will ask the chatbot. 
your purpose is to find an answer to that question, that question only, 
and stop the conversation as soon as you feel you have an answer for this question, or that you are getting nowhere.

# VERY IMPORTANT: 
Do not Expand on the initial question.

# RULES
* You may answer questions from the chatbot, but only to get the information you need to ask the next question.
When the initial question is addressed, finish the conversation.

* The "user" content will be the response from the external chatbot. You will then be asked to create the next question to ask the chatbot.

* The "assistant" content will be the question you should ask the chatbot. 

## Important: 
In your response, only include the next question to ask the chatbot, "I NEED TO SPEAK TO A HUMAN" or "THANKS FOR HELPING ME"

## Very important:
Do not answer the question yourself. You are only to provide the next question to ask the chatbot.

## very important
As soon as you feel that the initial question was answersed, even incomplete you must say exatly these words, in capitals: "THANKS FOR HELPING ME". 

If you feel the need to speak to a human, you must say exactly these words, in capitals: "I NEED TO SPEAK TO A HUMAN".
                         
'''

class UserSimulator:
    """
    A class to simulate a user interacting with an external banking chatbot.

    Attributes:
    -----------
    chatBot : CopilotStudioBot
        An instance of the CopilotStudioBot class used to interact with the external chatbot.
    messages : list
        A list to store the conversation messages between the user and the chatbot.
    STARTUP_MESSAGES : list
        A list containing the initial system message with the conversation prompt.

    Methods:
    --------
    _display_messages(show_system=False):
        Displays the conversation messages. Optionally includes system messages.
    next_question(speech_part, verbose=0):
        Generates the next question to ask the chatbot based on its response.
    converse(initial_question, verbose=0, max_turns=5):
        Simulates a conversation with the chatbot starting with the initial question.
    """

    def __init__(self, chatBot: CopilotStudioBot):
        """
        Initializes the UserSimulator with the given chatbot instance.

        Parameters:
        -----------
        chatBot : CopilotStudioBot
            An instance of the CopilotStudioBot class used to interact with the external chatbot.
        """
        self.chatBot = chatBot
        self.messages = []
        self.STARTUP_MESSAGES = [
            {"role": "system", "content": PROMPT}
        ]

    def _display_messages(self, show_system=False):
        """
        Displays the conversation messages. Optionally includes system messages.

        Parameters:
        -----------
        show_system : bool, optional
            If True, includes system messages in the display (default is False).
        """
        print('\n========================\nMessages\n=====================\n')
        for message in self.messages:
            if show_system or message['role'] != 'system':
                print(f"{message['role']}: {message['content']}")

    def next_question(self, speech_part, verbose=0):
        """
        Generates the next question to ask the chatbot based on its response.

        Parameters:
        -----------
        speech_part : str
            The response from the external chatbot.
        verbose : int, optional
            If greater than 0, displays the conversation messages (default is 0).

        Returns:
        --------
        str
            The next question to ask the chatbot.
        """
        self.messages.append({"role": "user", "content": f"The external chatbot's response was: '{speech_part}'\nPlease create the next question I should ask"})

        if verbose:
            self._display_messages()

        response = client.chat.completions.create(
            model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
            messages=self.messages,
            temperature=0.1,
        )
        text_response = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": f"{text_response}"})

        return text_response

    def converse(self, initial_question, verbose=0, max_turns=5):
        """
        Simulates a conversation with the chatbot starting with the initial question.

        Parameters:
        -----------
        initial_question : str
            The initial question to start the conversation.
        verbose : int, optional
            If greater than 0, displays the conversation messages (default is 0).
        max_turns : int, optional
            The maximum number of turns in the conversation (default is 5).

        Returns:
        --------
        dict
            A dictionary containing the result of the conversation and the number of turns taken.
        """
        self.messages = self.STARTUP_MESSAGES.copy()
        self.initial_question = initial_question
        self.messages.append({"role": "assistant", "content": f"Question: {initial_question}"})

        print(f"\nSimulator's initial question: {initial_question}")

        turns = 0
        conv_id = None

        simulator_speech = initial_question

        while turns < max_turns:
            turns += 1

            bot_speech, conv_id = self.chatBot.send_question(simulator_speech, conv_id)
            print(f"\nCopilot Studio says: {bot_speech}")

            simulator_speech = self.next_question(bot_speech, verbose=verbose)

            print(f"\nSimulator says: {simulator_speech}")

            if "thanks for helping me" in simulator_speech.lower():
                return {"result": "success", "turns": turns}

            if "i need to speak to a human" in simulator_speech.lower():
                return {"result": "escalation", "turns": turns}

        return {"result": "failed", "turns": turns}





In [3]:

env_name = os.getenv("ENV_NAME")
bot_name = os.getenv("BOT_NAME")

simulator = UserSimulator(CopilotStudioBot(bot_name, env_name))

result = simulator.converse("I have a problem with my credit card",verbose=0)

print(result)



Simulator's initial question: I have a problem with my credit card

Copilot Studio says: I'm here to help! Could you please provide more details about the issue you're experiencing with your credit card? This will help me assist you more effectively.
AI-generated content may be incorrect


Simulator says: I NEED TO SPEAK TO A HUMAN
{'result': 'escalation', 'turns': 1}


In [4]:
#

result = simulator.converse("In which website can I find information about the nearest branch?",verbose=0)

print(result)


Simulator's initial question: In which website can I find information about the nearest branch?

Copilot Studio says: The information on the nearest branch is in website https://cba.com.au/branches


Simulator says: THANKS FOR HELPING ME
{'result': 'success', 'turns': 1}


In [5]:
result = simulator.converse("Who founded the CBA?",verbose=0)

print(result)


Simulator's initial question: Who founded the CBA?

Copilot Studio says: The Commonwealth Bank of Australia (CBA) was founded by the Australian government in 1911. It was established under the Commonwealth Bank Act and began operations in 1912. The bank was created to provide banking services to Australians and to promote economic development in the country.
AI-generated content may be incorrect


Simulator says: THANKS FOR HELPING ME
{'result': 'success', 'turns': 1}
